# Forgetting, outputs and activations

To start with, I don't really know what I am looking for, just looking. I feel like I need to say that somewhere. 

Just get some outputs, and play around with pruning levels until it looks like something.

I can get everything I need from the first 5 cells of code, I automated some of it, but deleted the loop.

Change the targetted class with the remove_classes[] line in the split loader initialization.

In [1]:
import numpy as np
import cupy as cp

# This is needed to load the MNIST files for training and testing
import pickle

# These are the Sparana objects needed to build a model
from sparana.model import model
from sparana.layers import full_relu_layer
from sparana.layers import full_linear_layer
from sparana.optimizer import adam_optimizer
from sparana.data_loader import split_loader

# This is not needed to train and test a simple model, but I am going to demonstrate it here too
from sparana.saver import model_saver

# These are the extra imports mentioned above
from sparana.lobotomizer import lobotomizer
import os

# Put your own path in here
path = 'c:/users/jim/tensorflowtrials'

# This initializes the model object, the 2 things that are required are the input size, and a list of layer objects.
mymodel = model(input_size = 784, 
                # These are the layers input as a list, the final layer size is the number of classes the model will have.
                layers = [full_relu_layer(size = 1000), 
                          full_relu_layer(size = 800),
                          full_relu_layer(size = 400),
                          full_linear_layer(size = 10)],
                # This is set automatically, but I keep it in here as a demonstration
                comp_type = 'GPU')

# Initialize the weights here, after this randomly generated matrices are now in the GPU memory
mymodel.initialize_weights('Xavier', bias_constant = 0.1)

# Initialize the Adam optimizer, the associated matrices are now in GPU memory
opt = adam_optimizer(mymodel, 0.0001, epsilon = 0.001)

#Initialize the saver
mysaver = model_saver(mymodel)

Initalizing GPU weights


In [2]:
myloader = split_loader(pickle.load(open('MNIST_train_images.p', 'rb')),
                        pickle.load(open('MNIST_train_labels.p', 'rb')),
                        pickle.load(open('MNIST_test_images.p', 'rb')),
                        pickle.load(open('MNIST_test_labels.p', 'rb')),
                        remove_classes = [8],
                        maintain_classes = True)
# Load the model from pickle file.
mysaver.load_model('layers_and_pruning_100000_steps.p')
print(mymodel.get_accuracy(myloader.test_data(), myloader.test_labels()))
mysaver.store_model()

0.9858187458453357


In [3]:
# Initialize the lobotomizer

lobo = lobotomizer(mymodel)

# Load the data to be passed through

main_data, main_labels = myloader.random_minibatch(500)
removed_data, removed_labels = myloader.removed_minibatch(500)

# This is the command that does the work

lobo.MAAV_difference(main_data, removed_data)

# Same shit lol

for i in lobo._weight_stats:
    i.data = -i.data

# Start with looking at outputs

I know what I am looking at here, no extra steps. I can look at 10 output classes. 

In [8]:
mysaver.restore_model()
lobo.prune_smallest(layers = [0, 30000, 30000], print_stats = False, zero_stats = False)
data, labels = myloader.removed_test_set()
# The zero stats needs to be done here because I am inverting the vales
print(mymodel.get_accuracy(myloader.test_data(), myloader.test_labels()))
print(mymodel.get_accuracy(data, labels))
main = mymodel.outputs(myloader.test_data()[:5])
target = mymodel.outputs(data[:10])

0.9809439397296699
0.059548254620123205


In [9]:
np.set_printoptions(suppress=True)
print (np.round(main, 3))
print (np.round(target, 3))
print(np.mean(np.sum(main, axis = 1)))
print(np.mean(np.sum(target, axis = 1)))

[[-0.007  0.006  0.012  0.029 -0.002  0.002  0.011  0.945 -0.011  0.013]
 [-0.007  0.041  0.792  0.04  -0.012  0.071  0.085  0.046  0.055 -0.023]
 [-0.017  0.954  0.02   0.02   0.007 -0.01  -0.018 -0.    -0.009  0.021]
 [ 0.817 -0.005  0.048  0.035  0.022  0.006  0.053  0.042  0.034  0.017]
 [ 0.011  0.012  0.028  0.021  0.814  0.03   0.033  0.033  0.008  0.023]]
[[ 0.143  0.058  0.336  0.135  0.149  0.052  0.035  0.065  0.05   0.072]
 [ 0.093 -0.    -0.014  0.007  0.288  0.145  0.015 -0.084  0.241  0.367]
 [ 0.099  0.043  0.119  0.245  0.07   0.133  0.017  0.093  0.112  0.056]
 [ 0.175 -0.012  0.176  0.063  0.186  0.072 -0.022  0.039  0.104  0.275]
 [ 0.221  0.018  0.125  0.063  0.06   0.206  0.066  0.065  0.058  0.22 ]
 [ 0.233 -0.019  0.034  0.111  0.004  0.128  0.072  0.001  0.151  0.387]
 [ 0.296 -0.018  0.055  0.003  0.116  0.226  0.035 -0.     0.124  0.246]
 [ 0.173  0.125  0.305 -0.026  0.053  0.277 -0.045  0.063  0.069  0.039]
 [ 0.216  0.037  0.033 -0.004  0.074  0.181  0.055

# Sums


 | Zero | 100 | 500 | 1000 | 5000 | 10000 | 15000
---|---|---
**Target Sum**| 1.001 | 0.952 | 1.036 | 1.054 | 1.093 | 1.069 | 1.15
**Target Accuracy**|98.24% | 96.78% | 84.14% | 68.67% | 18.57% | 7.89% | 4.37%
**Main Sum**|1.0012 | 1.001 | 1.009 | 1.01 | 1.023 | 1.039 | 1.069
**Main Accuracy**| 98.59% | 98.65% | 98.61% | 98.59% | 98.55% | 98.45% |98.22%

Taking the means, this will be a small one, so it seems like a good place to put the accuracies, its not that clear from the a plot and it can't hurt. 

Should this really be a table? I have tables for everything else, onaroll. 

# Effect on main outputs

# Eights

Not going to put it in here, there is a difference, but that I will need to look further into that later. A difference in sums too. 

# Sevens

REally does not change. Not sure how to show this. 
# 0

Zero|
---|---|---
-0.005 | 0.001 | 1.027 | -0.007 | -0.003 | -0.005 | -0.003  |-0.001 | -0.001 | -0.
-0.001 | 0.998 |-0.002 |-0.001 | 0.002| -0.001 |-0. |    0.002 | 0.002| -0.
1.026 |-0.001 |-0.003 |-0.005 |-0.002 |-0.004 |-0.002 |-0.001 |-0.004 |-0.002
0.001 | 0.004 | 0.004 | 0.003 | 0.972 | 0.003 | 0.003 | 0.006 | 0.004 | 0.004
-0.001 | 1.001 |-0.001 | 0.001 | 0.  |  -0.002 | 0.  |   0.001 | 0. |    0.

# 15k

15,000|
---|---|---
 0.011 | 0.003 | 1.009 | 0.006 | 0.036 |-0.002 | 0.002 |-0.002 |-0.006 | 0.018
-0.01 |  0.98 |  0.012 | 0.005 |-0.002 |-0.003 | 0.01  | 0.012 |-0.006 | 0.033
0.972 | 0.044 | 0.015 |-0.015 | 0.034 | 0.001 | 0.013 | 0.004 |-0.012 | 0.034
0.021 | 0.044 |-0.005 | 0.033 | 0.81  | 0.044 | 0.049 | 0.03 |  0.014 | 0.015
0.003 | 0.945 | 0.014 | 0.012 |-0.  |   0.015 | 0.013 | 0.026 |-0.001 | 0.042
 
 The 4th one drops a bit, might be an example that would be forgotten easier if it were targetted. 
 
 15,000 is a lot, compare this to the changes seen in the target class. 
 
 There is no doubt that this will affect the way the model behaves with these classes as well, the question is how much. 

# Effect before accuracy drops

Parameters Removed | Main Accuracy | Output Values
---|---|---
zero | 0.9824 | 0.988 | 1.021 | 0.95 | 1.002 | 1.062 | 1.012 | 1.076 | 0.97 | 1.03 | 1.029
10 | 0.9776 | 0.938 | 0.966 | 0.917 | 0.992 | 1.045 | 0.962 | 0.999 | 0.914 | 0.982 | 0.95
20 | 0.9766 | 0.935 | 0.955 | 0.917 | 0.987 | 1.03 | 0.96 | 0.99 | 0.911 | 0.976 | 0.947
50 | 0.9747 | 0.851 | 0.871 | 0.858 | 0.945 | 0.952 | 0.885 | 0.915 | 0.849 | 0.894 | 0.846
100 | 0.9684 | 0.753 | 0.767 | 0.818 | 0.915 | 0.879 | 0.821 | 0.863 | 0.797 | 0.807 | 0.825
200 | 0.9581 | 0.6 | 0.625 | 0.741 | 0.836 | 0.777 | 0.721 | 0.76 | 0.711 | 0.65 | 0.704
300 | 0.9406 | 0.557 | 0.594 | 0.683 | 0.811 | 0.73 | 0.666 | 0.743 | 0.685 | 0.612 | 0.644
400 | 0.9202 | 0.501 | 0.545 | 0.659 | 0.747 | 0.664 | 0.595 | 0.757 | 0.624 | 0.568 | 0.611
500 | 0.8715 | 0.46 | 0.508 | 0.649 | 0.693 | 0.634 | 0.56 | 0.74 | 0.579 | 0.532 | 0.591


Going to go with 7s here. Only the 7 output class, doesn't matter here what else the model might be outputting. 

The main accuracy is still at that odd point where it is higher than we started with. 

Pretty obvious here that there is a significant effect early. This is 10 datapoints. Clearly shows there is a difference, but big change overall.

# Target class goes down, other classes go up

Here I want to demonstrate that pruning does not just reduce the size of the output of the target class. It seems to confuse the model. 

# A seven

 | 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9
---|---|---
**Zero** |-0. |  0.002 | 0.003 | 0.001 | 0.001 | 0.001 | 0.002 | 0.988 | 0.003 | 0. 
**500** |0.194 | -0.041 | -0.054 | 0.251 | -0.073 | -0.098 | -0.065 | 0.456 | 0.036 | 0.443
**1000** |0.236 | -0.056 | 0.001 | 0.362 | -0.08 | -0.106 | -0.096 | 0.341 | 0.04 |  0.433 
**2000** |0.204 | -0.106 | 0.086 | 0.379 | -0.091 | -0.099 | -0.076 | 0.179 | 0.123 | 0.384

I start with a 7 here because this is what I started with last time. This one is forgotten quite quickly. Here it seems to think that this one is either a 0, 3, 7, or 9. In the last row, 8 seems to be another option, the model is becoming more confused. This particular example is showing results for a relatively small number of parameters being pruned, so I looked at a couple more. 

# An eight

 | 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9
---|---|---
**Zero** | 0.012 | 0.02 | 0.096 | 0.031 | 0.026 | 0.024 | 0.021 | 0.034 | 0.713 | 0.018
**500** | 0.003 | -0.016 | 0.313 | 0.141 | -0.033 | -0.036 | 0. | -0.008 | 0.618 | 0.071
**1000** | 0.111 | -0.036 | 0.294 | 0.174 | -0.046 | 0.013 | -0.045 | -0.063 | 0.543 | 0.111
**2000** | 0.217 | -0.048 | 0.196 | 0.218 | -0.059 | 0.013 | -0.082 | -0.107 | 0.512 | 0.103
**5000** | 0.25 | -0.09 | 0.241 | 0.203 | -0.039 | 0.047 | -0.048 | -0.099 | 0.367 | 0.128
**15000** | 0.334 | -0.037 | 0.315 | 0.159 | 0.029 | 0.016 | 0.038 | -0.028 | 0.085 | 0.101

The model forgot this one quickly. Not as quickly as the 7, but I will get into that in the next experiment. This one could be a 0, 2, 3, 8, or 9. This is different to the 7 above.


# A different eight

 | 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9
---|---|---
**Zero** |-0.002 | 0. |    0. |   -0.002 | 0. |   -0.004 |-0. |    0.  |   1.01 | -0.002
**500** |  0.025 |-0.06 | -0.015 | 0.068 |-0.016 |-0.026 | 0.033 |-0.085 | 0.861 | 0.249
**1000** | 0.094 |-0.064 |-0.052 | 0.085 | 0.07 |  0.015 | 0.011 |-0.134 | 0.773 | 0.245
**2000** | 0.093 |-0.113 |-0.081 | 0.068 | 0.065 | 0.131 | 0.06 | -0.183 | 0.693 | 0.209
**5000** | 0.174 |-0.122 |-0.011 | 0.023 | 0.05 |  0.13 |  0.035 |-0.159 | 0.642 | 0.219
**15000** | 0.154 |-0.078 |-0.018 |-0.014 | 0.245 | 0.092 | 0.062 |-0.111 | 0.436 | 0.303

This one still registers as an 8 when the accuracy for 8s is 26%. It also starts to think that this is a 4, which doesn't really register for the example above. 

# Activations

Leaving this in here because I feel like I need to address this somewhere. I was intending to look at activations also, but there turned out to be more than enough to investigate with outputs. It is also less clear what I could learn from activations. I should look into this at some point, but there are more useful things to do first. 

What I was going to look at was the number of activations in each layer that fire, or pass on a non zero value. I had a brief look and found that there was quite a big difference between different datapoints of the same class, both before and after pruning. 